# TRPO vs PPO vs GRPO: Reinforcement Learning Algorithms for LLM Fine-Tuning

## Introduction and Background
Large Language Models (LLMs) are typically trained in three phases: 
1. An unsupervised pre-training on massive text
2. A supervised fine-tuning (SFT) on task-specific or instruction-following data
3. A Reinforcement Learning stage (often RLHF: RL with Human Feedback) to align the model with human preferences[1]. 

In the RLHF phase, the LLM acts as an agent that generates responses (actions) to prompts (states) and receives feedback scores (rewards) from a reward model or humans[2]. This RL phase has become crucial for pushing model performance and alignment beyond what pre-training and SFT achieve[3].

In this guide, we explore three key policy optimization algorithms used in RL for training LLMs: **Trust Region Policy Optimization (TRPO)**, **Proximal Policy Optimization (PPO)**, and **Group Relative Policy Optimization (GRPO)**. We will break down each algorithm's theory and mathematics, provide heavily-commented Python pseudocode for clarity, compare their strengths and weaknesses, and discuss practical considerations like scaling on GPU clusters and available frameworks.

*Note: We assume familiarity with basic RL concepts (policy, reward, advantage, etc.) and deep learning, but we will explain each algorithm step-by-step. Short, commented code examples are included to illustrate the implementations.*

## Trust Region Policy Optimization (TRPO)

Trust Region Policy Optimization (TRPO) is a policy-gradient method introduced by Schulman et al. (2015) to improve training stability for RL agents. The key idea is to restrict each policy update to a small region around the current policy (a trust region) so that performance doesn't collapse after an update[4]. TRPO achieves this by adding a Kullback–Leibler (KL) divergence constraint on policy updates[5]. In effect, instead of taking unconstrained gradient steps that might drastically change the policy, TRPO finds an update that maximizes reward while keeping the new policy close to the old policy (in terms of KL divergence). This yields a theoretical guarantee of monotonic improvement under certain conditions[6].

### Key features of TRPO:
- **Surrogate objective with advantage:** TRPO maximizes a surrogate objective. Here $A$ is the advantage of action $a$ in state $s$ under the old policy (i.e. how much better the action was than the old policy's baseline). This objective essentially reweights advantages by the probability ratio of new vs. old policy[7].
- **Trust region via KL constraint:** Instead of a fixed learning rate, TRPO enforces $KL(\pi_{old} || \pi_{new}) < \delta$ for some small $\delta$ [5]. Intuitively, this means "the new policy should not deviate too much from the old one" in one update. This hard constraint is handled by solving a constrained optimization (often via the conjugate gradient method to compute a trust-region step)[8][9].
- **Second-order optimization:** TRPO uses an approximate natural gradient step. It computes the policy gradient, then adjusts it by the inverse of the Fisher information matrix (related to the Hessian of the KL) to find the largest step that satisfies the KL constraint[9]. A line search is often performed to ensure the KL constraint and performance improvement criteria are met[10]. This second-order approach is more computationally intensive than first-order methods like standard policy gradients or PPO[11].
- **Actor-Critic architecture:** In practice TRPO (like most advanced policy gradient methods) uses a separate value function (critic) to estimate the baseline (expected return) for advantage calculation[12]. This means training an extra value network alongside the policy. Using a learned critic reduces variance of policy gradient estimates but adds complexity and memory cost[13].
- **Stability:** By limiting each update's size, TRPO provided much more stable training than earlier methods (like REINFORCE or simple Actor-Critic) and guaranteed (theoretical) monotonic policy improvement[6]. However, these benefits come at the cost of complexity and compute.

**Why TRPO is less common now:** TRPO was a breakthrough, but it is computationally expensive for deep neural network policies. The conjugate-gradient and line search procedure can be slow and hard to parallelize fully on GPUs. Indeed, TRPO's “very high” computational cost limits its use in modern large-scale settings[14]. It is rarely used for LLM training today because large models make the second-order updates impractical[15]. Simpler variants like PPO have become the go-to approach for most applications. Still, understanding TRPO is useful, since PPO and GRPO build on similar ideas.

### TRPO Pseudocode Example
Below is a simplified pseudocode for one iteration of the TRPO algorithm. It assumes we have a policy network $\pi_\theta$ (with parameters $\theta$) and a value (critic) network $V_\phi$. We first gather some trajectories (state-action-reward sequences) using the current policy, then compute advantages via the critic, and finally perform a TRPO policy update with a constrained optimization. Comments are included to explain each step:

In [1]:
import numpy as np
import torch

# Assume we have:
# policy: a neural network for policy pi_theta(s) that can compute log probabilities of actions.
# value_fn: a neural network V_phi(s) to predict state values for baseline.

# Hyperparameters for TRPO
max_kl = 0.01         # KL divergence limit (trust region size)
cg_iters = 10         # iterations for conjugate gradient
backtrack_iters = 10  # line search iterations for step size
backtrack_coeff = 0.8 # step size reduction factor in line search

# Step 1: Collect trajectories using current policy
trajectories = collect_trajectories(policy, env, num_trajectories=50, horizon=500)
# Each trajectory contains (states, actions, rewards) for an episode or batch of steps.

# Step 2: Compute returns and advantages for each time step in each trajectory
for traj in trajectories:
    rewards = traj["rewards"]
    states  = traj["states"]
    # Compute discounted cumulative returns G_t for each step t
    returns = compute_discounted_returns(rewards, gamma=0.99)
    # Get state value estimates from critic for baseline
    values  = value_fn(states)
    # Advantage = return - value (could use GAE for smoothing advantages)
    advantages = returns - values.detach().numpy()
    traj["advantages"] = advantages
    traj["returns"]    = returns

# Combine trajectory data for training
states     = np.concatenate([traj["states"] for traj in trajectories])
actions    = np.concatenate([traj["actions"] for traj in trajectories])
advantages = np.concatenate([traj["advantages"] for traj in trajectories])
old_log_probs = policy.get_log_prob(states, actions).detach()  # log π_old(a|s)

# Step 3: Compute policy gradient under the current policy
# This is the gradient of the surrogate objective L = E[ (pi_theta / pi_old) * A ]
policy_loss = -torch.mean(torch.exp(policy.get_log_prob(states, actions) - old_log_probs) * torch.tensor(advantages))
policy_grad = torch.autograd.grad(policy_loss, policy.parameters())
policy_grad = flat_concat(policy_grad)  # flatten gradient vector

# Step 4: Define a function to compute KL and Fisher-vector product for conjugate gradient
def compute_kl_and_fisher_vec(vec):
    # Compute KL divergence between current policy and a hypothetical policy moved by vector 'vec'
    assign_params(policy, theta_current + vec)        # temporarily shift policy by vec
    new_log_probs = policy.get_log_prob(states, actions)
    kl = torch.mean(torch.exp(old_log_probs) * (old_log_probs - new_log_probs))  # KL(π_old || π_new)
    # Compute gradient of (∇_theta KL * vec) which gives Fisher-vector product
    kl_grad = torch.autograd.grad(kl, policy.parameters(), create_graph=True)
    kl_grad = flat_concat(kl_grad)
    fisher_vec_product = torch.autograd.grad(torch.dot(kl_grad, torch.tensor(vec)), policy.parameters())
    fisher_vec_product = flat_concat(fisher_vec_product)
    assign_params(policy, theta_current)              # restore original parameters
    return kl.item(), fisher_vec_product

theta_current = get_flat_params(policy)  # current policy parameters (flattened)
# Conjugate Gradient to solve F * x = -g (find search direction x = F^{-1} * (-grad))
g = -policy_grad  # we want to maximize the objective, so take negative grad for ascent
x = conjugate_gradient(lambda v: compute_kl_and_fisher_vec(v)[1], g, cg_iters)

# Step 5: Determine step length to satisfy KL constraint
# Compute step direction and scale it to meet KL limit
step_dir = x  # this is the proposed natural gradient step
# Compute KL for full step
current_kl, _ = compute_kl_and_fisher_vec(step_dir)
if current_kl > max_kl:
    # Scale step to meet KL constraint: scale = sqrt(max_kl / current_kl)
    step_dir *= np.sqrt(max_kl / (current_kl + 1e-8))

# Step 6: Line search along step direction to ensure improvement
step_size = 1.0
for _ in range(backtrack_iters):
    new_params = theta_current + step_size * step_dir
    assign_params(policy, new_params)
    # Surrogate objective at new params
    new_loss = -torch.mean(torch.exp(policy.get_log_prob(states, actions) - old_log_probs) * torch.tensor(advantages))
    # Compute KL divergence between new and old policy
    kl, _ = compute_kl_and_fisher_vec(np.zeros_like(theta_current))  # KL at new_params vs old (vec=0 since policy already updated)
    if new_loss.item() < policy_loss.item() and kl < max_kl:
        # Found acceptable update
        theta_current = new_params
        break
    step_size *= backtrack_coeff  # reduce step and try again
# End line search

# Step 7: Update policy to new parameters
assign_params(policy, theta_current)

# Step 8: Update value network (critic) by regression to the returns
critic_loss = torch.mean((value_fn(states) - torch.tensor(returns))**2)
critic_optimizer.zero_grad()
critic_loss.backward()
critic_optimizer.step()

ModuleNotFoundError: No module named 'torch'

**Explanation:** In the code above, we:
1. Collect trajectories with the current policy.
2. Compute returns (discounted sums of rewards) and advantages for each state-action using a baseline from the value network.
3. Compute the policy gradient of the surrogate objective. We use importance sampling (the ratio of current policy probability to old policy probability) multiplied by the advantage.
4. Use a conjugate gradient solver to compute the natural gradient step direction $x = F^{-1}g$ where $F$ is the Fisher information matrix (from the KL) and $g$ is the policy gradient[9].
5. Scale the step to satisfy the KL divergence constraint ($KL < \delta$), then perform a line search to ensure the update actually improves the objective and respects the KL limit[10].
6. Update the policy parameters by this constrained step.
7. Finally, update the value network (critic) via supervised learning on the observed returns.

TRPO's implementation is complex, as seen above. Modern frameworks (e.g. OpenAI SpinningUp) provide reference implementations[16]. Due to this complexity, simpler algorithms like PPO have been more widely adopted for large-scale problems.

## Proximal Policy Optimization (PPO)

Proximal Policy Optimization (PPO) is a family of first-order policy gradient methods introduced by Schulman et al. (2017) as a simpler alternative to TRPO that achieves comparable performance. PPO has become the de facto RL algorithm for many applications, including fine-tuning large language models (ChatGPT, Google's Gemini, etc.)[17]. It maintains the core idea of limiting policy updates to avoid instability, but does so with a simpler mechanism: clipped probability ratios instead of an explicit KL constraint[18]. This avoids the need for second-order optimization and makes PPO much easier to implement and parallelize on GPUs.

### Key features of PPO:
- **Clipped surrogate objective:** PPO defines a surrogate loss similar to TRPO’s, but instead of a hard KL constraint, it clips the policy change. Define the probability ratio $r(\theta) = \frac{\pi_\theta(a|s)}{\pi_{old}(a|s)}$. The PPO objective to maximize is:
  $$L_{\text{PPO}}(\theta) = \mathbb{E}_{s,a}\Big[ \min\big( r(\theta)A, \text{clip}(r(\theta), 1-\epsilon, 1+\epsilon)A \big) \Big]$$
  where $A$ is the advantage estimate and $\epsilon$ is a small hyperparameter (e.g. 0.1 or 0.2) controlling the clip range[19][20]. This formulation means: if $r(\theta)$ tries to push the policy far from the old policy (outside $[1-\epsilon, 1+\epsilon]$), it gets clipped, effectively discarding extreme policy updates. The optimizer then maximizes the minimum of the unclipped and clipped objectives, ensuring that improvements are limited to a “proximal” range.
- **First-order, easily implemented:** PPO uses standard stochastic gradient ascent (or descent on a defined loss) with the clipped objective. It requires only first-order gradient computation – no conjugate gradient, no line search. This makes it much faster and easier to implement than TRPO while achieving similar stabilizing effects[18]. PPO can be run with multiple epochs over each batch of data and works well with large neural networks on GPUs.
- **Actor-Critic with value function:** Like TRPO, PPO typically uses an actor-critic approach. The policy (actor) is updated with the PPO loss, and simultaneously a value network (critic) is trained (usually with a mean-squared error loss) to predict state values for advantage estimation (often using Generalized Advantage Estimation to get a low-variance, somewhat biased advantage)[21]. This again means maintaining a separate head or network for value estimates, doubling the model parameters in many implementations[13]. (In practice for LLMs, the value "network" might share most parameters with the policy, but it still introduces extra memory and compute overhead for the value head.)
- **Strong empirical performance:** PPO has been remarkably successful across many domains. It achieved state-of-the-art results in games (e.g. OpenAI Five for Dota2) and robotics, and has been the workhorse algorithm for RLHF in language models (used by OpenAI in training ChatGPT, by DeepMind, etc.)[17]. It finds a sweet spot of being simpler than TRPO yet robust against the destructively large policy updates that vanilla policy gradient might produce[22].
- **Tuning and variants:** PPO has a few variants (e.g. PPO with KL penalty instead of clipping, sometimes called PPO2, and many tweaks in practice). But the clipped PPO described above is most common. It has hyperparameters like clip range $\epsilon$, number of epochs, advantage estimation parameters, etc., which generally are easier to tune than TRPO's parameters.

### PPO Training Loop Example
Below we illustrate a simplified training loop for PPO using pseudocode, highlighting the key steps. We’ll use a made-up environment and policy for demonstration. This code would typically run for many iterations, but we show one iteration for clarity. Comments explain each part of the process:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assume we have:
# policy: a neural network mapping state -> action distribution (actor)
# value_fn: a neural network mapping state -> value (critic)
# Both share parameters or not depending on implementation.

policy_optimizer = optim.Adam(policy.parameters(), lr=3e-4)
value_optimizer  = optim.Adam(value_fn.parameters(), lr=1e-3)
clip_ratio = 0.2   # PPO clip parameter epsilon
train_epochs = 4   # how many epochs to train on one batch of trajectories
batch_size  = 64   # minibatch size for each epoch

# Collect trajectories (batch of experience) using current policy
batch = collect_batch(policy, env, batch_size=2048, max_steps=200)
# batch contains tensors: states, actions, rewards, next_states, done, old_log_probs (log π_old(a|s))

# Compute rewards-to-go and advantages using the current value function
with torch.no_grad():
    # Compute value estimates for all states and next_states
    values      = value_fn(batch.states)
    next_values = value_fn(batch.next_states)
# Compute Generalized Advantage Estimation (GAE) or simple advantage
advantages, returns = compute_gae_advantages(batch.rewards, batch.dones, values, next_values, gamma=0.99, lam=0.95)
advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)  # normalize advantages for stability

# Add the advantage and returns to the batch
batch.advantages = advantages
batch.returns    = returns

# Record old policy probabilities (from data collection) for importance sampling
old_log_probs = batch.old_log_probs

# PPO policy and value network training
for epoch in range(train_epochs):
    # Shuffle and iterate over mini-batches
    for minibatch in batch.iter_minibatches(size=batch_size):
        # Extract mini-batch data
        mb_states      = minibatch.states
        mb_actions     = minibatch.actions
        mb_advantages  = minibatch.advantages
        mb_returns     = minibatch.returns
        mb_old_logp    = minibatch.old_log_probs

        # Forward pass: get current log probabilities and entropy from policy
        dist = policy(mb_states)               # get action distribution for current states
        logp = dist.log_prob(mb_actions)       # log π_theta(a|s) for taken actions
        # Also get value function predictions for these states
        values_pred = value_fn(mb_states).squeeze(-1)

        # Calculate the probability ratio r(θ) = exp(log π_new - log π_old) for each action
        ratio = torch.exp(logp - mb_old_logp)
        # Compute unclipped and clipped policy advantages
        unclipped_obj = ratio * mb_advantages
        clipped_ratio = torch.clamp(ratio, 1 - clip_ratio, 1 + clip_ratio)
        clipped_obj   = clipped_ratio * mb_advantages
        # PPO policy loss: maximize the minimum => minimize negative of it
        policy_loss = -torch.mean(torch.min(unclipped_obj, clipped_obj))
        # Optional: add an entropy bonus to encourage exploration (not shown here for brevity)

        # PPO value function loss: mean squared error of value predictions vs returns
        value_loss = torch.mean((values_pred - mb_returns)**2)

        # Take optimization steps for policy and value networks
        policy_optimizer.zero_grad()
        policy_loss.backward()
        policy_optimizer.step()

        value_optimizer.zero_grad()
        value_loss.backward()
        value_optimizer.step()

**Explanation:** This pseudocode outlines a typical PPO training iteration:
1. We collect a batch of experiences (multiple timesteps from multiple actors or environment rollouts) using the current policy. During collection, we store the log probability of each action under the current (old) policy – this will be used to compute the importance weight ratio $r(\theta)$.
2. We compute the advantage estimates for each time-step. Here we show using GAE (Generalized Advantage Estimation) which uses the value function to get a smoother advantage. The advantage is normalized (zero mean, unit variance) for numerical stability.
3. Then we perform several epochs of stochastic gradient descent on this batch:
    - For each minibatch of data, we compute the current policy's log-probabilities for the actions. The ratio $r(\theta)$ is calculated.
    - We compute the clipped objective: the code above forms `unclipped_obj = ratio * advantage` and `clipped_obj = clipped_ratio * advantage` (where `clipped_ratio` is $r$ clipped to $1 \pm \epsilon$). The policy loss is the negative mean of `min(unclipped_obj, clipped_obj)` — this implements the PPO objective which penalizes too-large policy updates by taking the lower bound of the unclipped and clipped advantage terms[22].
    - We also compute a value loss (squared error between predicted value and the computed return) to update the critic.
    - Then we backpropagate and update both networks (policy and value) with gradient descent. This loop is repeated for a few epochs over the same data to improve data efficiency.

PPO's clipped update makes it stable (prevents big jumps in policy) while forgoing TRPO’s complicated constraint solving. It has become a standard algorithm due to this balance of stability and simplicity[18]. Indeed, “PPO is now the preferred approach in most LLM architectures, including ChatGPT, Gemini, and more”[17]. However, one downside in the context of LLM fine-tuning is that maintaining a separate value head or network can be memory-intensive for very large models (you effectively have two large networks to train)[13]. This is where GRPO enters the stage, aiming to simplify the architecture.

## Group Relative Policy Optimization (GRPO)

Group Relative Policy Optimization (GRPO) is a recent algorithm (2023–2024) introduced by the DeepSeek team (Shao et al., 2024) to further improve PPO, specifically targeting large language model fine-tuning for complex tasks like mathematical reasoning[23][24]. GRPO was designed to boost LLM reasoning performance while cutting down memory and compute needs[13]. In essence, GRPO keeps the core ideas of PPO (policy gradients with clipped updates for stability) but makes a crucial change: it eliminates the separate value network (critic)[25][26]. Instead, it computes advantages in a novel way using grouped experiences – hence “Group Relative”.

### Key innovations of GRPO:
- **No Critic (No value network):** Unlike TRPO and PPO which train a value function to serve as a baseline, GRPO drops the value network entirely[27]. This means we no longer double the model parameters with a critic. The policy itself is the only network to train for decision-making. This change dramatically reduces resource usage – in practice, DeepSeek reported ~40% reduction in peak GPU memory and faster training (only one forward/backward pass per update instead of two)[28]. By simplifying the architecture, GRPO “cuts memory and compute in half” without sacrificing performance[13].
- **Group-based advantage estimation:** How can we estimate the advantage without a value function? GRPO's solution: for each query (state), generate a group of responses (actions) in parallel and use their relative rewards as the baseline. Concretely, if for a given prompt (state) we sample $G$ different responses using the current policy, and each response $i$ gets a reward $r_i$ (from the reward model or environment), we can compute an advantage for response $i$ as a normalized difference from the group average. For example, one approach is to compute the advantage $A_i$ as a z-score within that group[29]:
  $$A_i = \frac{r_i - \mu_{\text{group}}}{\sigma_{\text{group}} + \epsilon}$$
  where $\mu_{\text{group}}$ is the mean reward of the group and $\sigma_{\text{group}}$ is the standard deviation of rewards in the group (and $\epsilon$ a small constant for stability). This basically measures how much better (or worse) a response’s reward is compared to other responses for the same prompt[29]. If $r_i$ is higher than the group average, the advantage is positive; if lower, the advantage is negative. Using the std-dev to scale is optional (ensuring a unit-variance normalization); in practice one can scale by group or even across the whole batch[30]. An even simpler variant is a leave-one-out baseline: $A_i = r_i - \frac{1}{G-1}\sum_{j \neq i} r_j$ [31], which is conceptually very similar (this yields the same formula as above up to a constant factor).
- **Why this works:** The group’s average reward serves as an estimate of “expected reward” for that prompt, akin to a value baseline[32]. By comparing each outcome to others for the same prompt, we get a high-quality advantage estimate without training a value predictor. This approach leverages the fact that LLMs can generate many parallel outputs for the same input by sampling with different randomness (for example, using a low temperature to introduce variability)[32]. Many RLHF pipelines already generate multiple samples per prompt to choose the best; GRPO turns this into an advantage computation strategy.
- **Clipped PPO-style update:** GRPO uses the same kind of clipped objective as PPO when updating the policy[33]. Once we have the advantages for each sampled action, we compute the policy loss exactly like PPO: e.g. $\min(r(\theta)A, \text{clip}(r(\theta), \dots)A)$. In other words, it keeps the “trust region” behavior of PPO’s clamp on the policy ratio, ensuring updates are not too large[33]. This retention of PPO’s core makes GRPO’s updates just as stable as PPO’s, guarding against training instability or divergence[33]. Empirically, GRPO achieves smooth and monotonic improvement similar to PPO[33].
- **KL regularization to a reference model:** Another tweak in GRPO (especially for RLHF on LLMs) is adding a KL divergence penalty comparing the current policy to a reference policy (usually the pre-trained or SFT model)[34]. In RLHF, it's common to keep the fine-tuned policy from straying too far from the original model's behavior (to avoid nonsensical outputs). GRPO includes this by directly adding a term to the loss like $\beta D_{KL}(\pi || \pi_{ref})$, where $\pi_{ref}$ is the reference model (e.g., the SFT model)[34]. This is similar to how earlier approaches added a KL penalty into the reward or objective, but GRPO's formulation treats it explicitly in the loss. The result is the policy is gently regularized to stay close to the original distribution unless the reward incentive strongly pulls it away. In practice, this helps prevent the RL finetuning from degrading the model’s language quality or diverging off-distribution while still allowing improvement on the reward metric.
- **Efficiency and performance:** By eliminating the critic, GRPO not only simplifies training but also frees up a lot of memory and compute. The DeepSeek team notes that “eliminating the heavyweight value model slashes peak GPU memory by ~40%, and training time drops because there’s only one backward pass per update (policy alone) instead of two”[28]. This means larger batch sizes or models can be used on the same hardware budget, or simply faster turnaround. Despite being leaner, GRPO matches or even exceeds PPO in fine-tuning performance on challenging tasks. For example, a 1.5B parameter code-generation model fine-tuned with GRPO saw its code compile success rate jump from ~60% to ~80%, outperforming the PPO-tuned baseline[35]. On a mathematical reasoning benchmark (Maze navigation tasks), adding GRPO fine-tuning improved accuracy from 86% to 93% with far fewer training steps than PPO[36]. These results show GRPO can yield superior results, particularly in tasks requiring reasoning, with lower resource usage.

In summary, GRPO brings the best of both worlds: PPO-like stability and performance, with a simpler, more hardware-efficient training loop[37]. It’s especially appealing for fine-tuning large LLMs where memory and speed are at a premium, and where generating multiple outputs per prompt is feasible.

### GRPO Training Pseudocode
Now, let's outline how a GRPO training step might look. We assume we have a language model policy (no separate value network) and a reward function (which could be a learned reward model or some automatic metric). We'll simulate one update where for each prompt in a batch, we generate multiple responses and compute the policy gradient. Comments highlight the differences from PPO:

In [ ]:
import torch
import torch.nn.functional as F

# Assume:
# policy_model is our language model (policy) that outputs a distribution over tokens.
# reward_model is a pretrained model that can score a (prompt, response) pair with a scalar reward.
# (In practice, reward_model could be a separate network trained from human feedback.)

batch_prompts = sample_prompts(batch_size=32)  # e.g., 32 different query prompts

G = 4  # number of responses to generate per prompt (group size)
responses = []  # to store all generated responses
log_probs = []  # to store log-probs of those responses under current policy
rewards = []    # to store reward model outputs

# Step 1: Generate G responses for each prompt using the current policy model
for prompt in batch_prompts:
    prompt_responses = []
    prompt_logps = []
    for _ in range(G):
        output = policy_model.generate(prompt, max_length=100, do_sample=True, temperature=0.7)
        # `generate` returns a sequence of tokens (the model's response).
        prompt_responses.append(output)
        # Compute log probability of the generated sequence under the current policy
        logp = policy_model.logprob(prompt, output)  # sum of log probs of each token in the sequence
        prompt_logps.append(logp.detach())
        # Score the (prompt, output) with reward model (e.g., a scalar preference score)
        R = reward_model.evaluate(prompt, output)
        rewards.append(R)
    # Store the group of responses and log-probs for this prompt
    responses.append(prompt_responses)
    log_probs.append(prompt_logps)

# Step 2: Compute advantages using group relative rewards
advantages = []
idx = 0
for i, prompt in enumerate(batch_prompts):
    # rewards for the i-th prompt's G responses:
    r_group = torch.tensor(rewards[idx: idx+G], dtype=torch.float32)
    idx += G
    # Compute group mean and (optionally) std
    mean_R = torch.mean(r_group)
    std_R  = torch.std(r_group)
    for r in r_group:
        # Advantage = (reward - mean) / (std + eps)
        if std_R > 1e-6:
            adv = (r - mean_R) / (std_R + 1e-8)
        else:
            adv = r - mean_R  # if zero variance, advantage is difference from mean
        advantages.append(adv)
advantages = torch.stack(advantages)  # shape [batch_size*G]

# Step 3: Prepare policy loss with PPO clipping (no value loss since no value network)
log_probs = torch.cat([torch.stack(lp) for lp in log_probs])  # flatten log_probs of all prompt responses
# For importance sampling, we need old log-probs. 
# In practice, we'd store the log_probs from when these responses were generated (the policy hasn't changed since generation here).
old_log_probs = log_probs.detach()  # treat current log_probs as old since we haven't updated yet

# Compute policy ratio r = exp(new_logp - old_logp). Here new and old are same (one update), 
# but if this loop iterates multiple epochs, we'd recompute new_logp each time.
new_log_probs = policy_model.logprob_batch(batch_prompts, responses)  # hypothetical function to get log-probs for all generated outputs
ratio = torch.exp(new_log_probs - old_log_probs)

# Clipped PPO objective with our advantages
clip_eps = 0.2
unclipped_obj = ratio * advantages
clipped_obj   = torch.clamp(ratio, 1-clip_eps, 1+clip_eps) * advantages
ppo_objective = torch.mean(torch.min(unclipped_obj, clipped_obj))  # we want to maximize this

# KL regularization: compare policy to reference (e.g., SFT model) on the generated outputs
# Compute KL(old_policy || reference_policy) as an additional penalty
ref_log_probs = reference_model.logprob_batch(batch_prompts, responses)
kl_div = torch.mean(torch.exp(old_log_probs) * (old_log_probs - ref_log_probs))
# (Above is an approximation: effectively average KL over tokens or sequences)

beta = 0.1  # weight for KL penalty
loss = -ppo_objective + beta * kl_div  # final loss to MINIMIZE (negative of objective plus KL term)

# Step 4: Update policy model via backpropagation
optimizer.zero_grad()
loss.backward()
optimizer.step()

**Explanation:** In this GRPO pseudocode, pay attention to what’s missing compared to PPO: there is no value function update at all. The steps are:
1. **Generate multiple trials per prompt:** For each prompt in the batch, we sample $G$ different responses using the current policy (the language model). This can be done in parallel on a GPU. We store the log-probabilities of these responses under the policy (for use in the loss) and obtain a reward for each (via a reward model or some metric). Now we have a set of $G$ tuples. For example, if batch_size = 32 and G = 4, we have 128 responses total, grouped by 32 prompts.
2. **Compute group-based advantages:** For each prompt’s group of responses, we compute the mean reward (and std). We then assign each response an advantage equal to reward minus the mean (optionally divided by std)[29]. This gives a zero-mean advantage within each group: responses better than average for that prompt get positive advantage, worse than average get negative. These advantages are all computed from actual rewards, with no learning involved. They naturally center the rewards per prompt, which reduces variance (akin to a baseline). This is essentially performing a relative reward comparison for each prompt.
3. **Policy optimization (PPO style):** We then treat all the collected responses as our data batch and perform a policy gradient update. We compute the probability ratios $r(\theta)$ for each (prompt, response) using the current policy vs. the stored log-prob (which is effectively the same in this single update case). We form the PPO clipped loss as usual, using our computed advantages. This yields the policy gradient encouraging the policy to increase probability of above-average responses and decrease probability of below-average ones.
4. **KL regularization:** We also compute a KL divergence between the current policy and a reference policy (such as the pre-trained model) on the same data. In the code, `kl_div` is computed as an average forward KL (old policy vs. reference) over the batch. We add a term $\beta D_{KL}$ to the loss (with $\beta$ a hyperparameter)[34]. This penalizes the policy for straying too far from the original model’s behavior and helps retain fluency and prevent overshooting.
5. **Update the policy:** Finally, we backpropagate the combined loss and update the policy model’s parameters. Notably, we did not need to do any value function update (no critic).

In practice, GRPO can be implemented to run multiple epochs on the collected batch (similar to PPO) if desired, and the KL penalty can be tuned or even turned off for certain tasks. But the core distinction is the advantage computation and the absence of a learned value baseline. This makes each iteration much lighter. The HuggingFace TRL library provides a high-level `GRPOTrainer` that encapsulates these steps for language model fine-tuning[38], making it easy to apply GRPO in real LLM training.

## Comparative Analysis: TRPO vs PPO vs GRPO

Now that we've detailed each algorithm, let's compare them in terms of requirements, efficiency, and use cases:

| Feature | TRPO | PPO | GRPO |
| :--- | :--- | :--- | :--- |
| **Algorithm Complexity** | High (Second-order, Conjugate Gradient) | Medium (First-order, Clipping) | Low (First-order, No Critic) |
| **Requires Critic?** | Yes (Value Network) | Yes (Value Network) | No (Group Relative Baseline) |
| **Stability** | Very High (Trust Region Constraint) | High (Clipped Updates) | High (Clipped Updates + Group Baseline) |
| **Efficiency** | Low (Slow updates, hard to parallelize) | High (Fast updates, parallelizable) | Very High (Fastest, memory efficient) |
| **Memory Usage** | High (Policy + Value Network) | High (Policy + Value Network) | Low (Policy Only) |
| **Primary Use Case** | Robotics, Games (Historical) | General RL, LLM RLHF (Standard) | LLM Reasoning, Large Scale RLHF |

### Strengths & Weaknesses:

**TRPO**
- **Strengths:** Very stable updates, theoretical guarantees of monotonic improvement, often needs fewer tuning of learning rate since trust region is automated.
- **Weaknesses:** Computationally heavy, not well-suited to large neural nets without modifications, complex implementation, and double networks.

**PPO**
- **Strengths:** Simplicity and ease of implementation, good stability in practice, widely tested and tuned, efficient on GPUs (just first-order gradients), flexible (works with many environments and can incorporate heuristics like entropy bonuses, etc.).
- **Weaknesses:** Still requires tuning (clip range, advantage normalization, etc.), and maintaining the critic can be tricky (e.g. if the value function lagging can cause high advantage variance). Also for huge models, the critic adds memory overhead.

**GRPO**
- **Strengths:** Highly efficient in memory and compute (especially for large models)[28], no need to train a value function (one less source of error and instability), leverages multiple samples to directly use reward feedback (which can improve learning signal for tasks like QA or coding where a single outcome can be definitively better than another). It maintains PPO’s clipped update reliability[33]. It's particularly good for tasks where relative ranking of multiple outputs is meaningful (like human preference comparisons or checking which of several answers is more correct).
- **Weaknesses:** Being newer, it's less battle-tested; it relies on having a reward signal that can compare multiple outcomes (needs a decent reward model or metric). Also, because it uses sequence-level advantage for each output, it might ignore token-level credit assignment (the value function in PPO gives per-token feedback, whereas GRPO gives the same advantage to all tokens in a generated response). In practice this hasn't proven a major issue for final performance, but it's a conceptual trade-off[45]. Additionally, if an environment doesn't easily allow multiple trials from the same state (unlike LLM prompts where you can sample many outputs), one must adapt GRPO (e.g. by aggregating experiences across similar states).

In the context of LLM training, PPO and GRPO are currently among the top choices. PPO has been the standard, but GRPO is gaining traction as training large models with RL becomes more common and resource-intensive. We also note there are non-RL or hybrid approaches like DPO (Direct Preference Optimization) (which directly optimizes the policy using ranked preferences without a reward model) – DPO similarly doesn't require a value network and has a simpler loss, but it optimizes for preference rankings rather than interactive environment return[46]. DPO has been used for aligning LLMs with human ranking data as well, and is quite stable and simple. However, pure RL approaches (like PPO/GRPO) can in principle optimize more complex long-horizon objectives and encourage exploration in ways supervised approaches cannot.

## Scaling to Large-Scale Hardware

Training reinforcement learning algorithms on large models (like LLMs with billions of parameters) requires substantial hardware. Here we discuss hardware-aware considerations and how TRPO, PPO, and GRPO each fit into large-scale GPU cluster training.

- **Parallelism:** Unlike supervised learning on a fixed dataset, RL involves a data generation loop (the agent must act in an environment or generate outputs to get rewards). For LLM fine-tuning, the "environment" is often just queries answered by the model and scored by a reward model. To speed up training, one typically generates experiences in parallel. Using multiple GPUs and even multiple nodes is essential to get enough throughput (for example, generating hundreds or thousands of samples per second). PPO and GRPO are well-suited to parallel rollout generation – you can have many processes or devices sampling responses concurrently, then aggregate the results for the update. TRPO, by contrast, is harder to parallelize because of its sequential optimization procedure (conjugate gradient and line search) which doesn't map as cleanly to a distributed setting. This is one reason PPO became popular: it fits a simple distributed actor-learner architecture (many actors collecting data, one learner updating the policy) easily.
- **Batched computations on GPU:** All three algorithms rely on computing policy network forward passes for many samples. Modern implementations use GPUs to vectorize these computations. PPO and GRPO benefit from being able to use large batch sizes (since gradient descent steps are simpler first-order operations). GRPO in particular can use the freed-up memory (from not having a value net) to hold more samples in a batch or a larger model[28], making it even more efficient on GPU. TRPO's heavy computation (e.g. large matrix-vector products for Fisher matrix) can be slow on GPU; though libraries can still leverage GPU for those, the algorithm inherently does more serial work per update.
- **Distributed training:** When scaling to multi-GPU and multi-node clusters, frameworks like Distributed Data Parallel (DDP) in PyTorch or Horovod, and high-performance communication (NCCL on NVIDIA GPUs over fast interconnects like InfiniBand), are used to synchronize gradients or share experience batches. PPO is commonly distributed by having each worker collect data and perform updates on its slice, or by gathering all data to a central learner. GRPO can similarly distribute the generation of groups of responses: each GPU can handle a subset of prompts and their G samples, then the advantages can be computed locally (or in a reduced way across nodes if using batch normalization of rewards). Because GRPO only needs to synchronize the policy parameters (like standard SGD) and perhaps the computed advantages, it fits well with data-parallel training across a cluster. InfiniBand or other high-bandwidth interconnects help ensure that the large model gradients (and possibly large tensors of samples) can be communicated quickly between nodes, keeping GPUs from idling. For example, training a 0.5B parameter model with GRPO on 8 GPUs was reported to take about 1 day[47] – scaling further to larger models or more GPUs would require efficient communication to maintain that speed.
- **Memory and model parallelism:** For very large models (billions of parameters), single GPU memory may be insufficient. Techniques like model parallelism (sharding the model across GPUs) or using optimized libraries (DeepSpeed, Megatron-LM) are needed. If using model parallelism, the overhead of multiple networks is even more acute – having to have both a policy and value model can effectively double the memory overhead across shards. GRPO's design directly helps here: only one model to parallelize. In a multi-node scenario, removing the critic means less cross-node communication of parameters and gradients. Additionally, GRPO's reduced memory footprint could allow enabling memory-heavy optimizations (like using a larger batch or sequence length) which can improve throughput on GPU clusters.
- **Throughput vs. sample efficiency:** In deep RL for games or robotics, sample efficiency (not needing too many environment steps) is often paramount. In LLM fine-tuning, we often care more about wall-clock time – we have a fixed dataset of queries or can generate as many as needed. Algorithms like PPO and GRPO are very throughput-optimized on hardware (lots of parallel sampling and fast updates), whereas TRPO might achieve slightly better learning per sample in some cases but lose out in wall-clock due to lower parallel efficiency. For large-scale training on clusters, simpler algorithms that make good use of GPUs usually win out. Indeed, GRPO’s creators explicitly prioritized making RLHF training faster and cheaper on real hardware by cutting out unnecessary computation[48].

In summary, for large GPU clusters (with high-speed interconnects like InfiniBand), PPO and GRPO scale effectively. GRPO, by virtue of reduced complexity, likely scales even better, as it was designed with modern LLM training constraints in mind. TRPO is generally not considered for LLMs at scale due to its computational demands not aligning well with straightforward GPU acceleration.

## Frameworks and Tools

Implementing these algorithms from scratch is complex, especially for large models. Fortunately, there are several frameworks and libraries that provide robust implementations:

- **OpenAI SpinningUp:** A great educational resource with reference implementations of many RL algorithms. Notably, it includes a clean PyTorch implementation of TRPO[16] and PPO. It's meant for learning and smaller-scale experiments.
- **Stable Baselines3 (SB3):** A popular Python library with reliable implementations of many RL algorithms (PPO, A2C, DQN, etc.) in PyTorch. SB3 (and its predecessor Stable Baselines) focus on ease of use for environments like Gym. The core SB3 library includes PPO (widely used), but not TRPO in the main package. TRPO is available in an add-on repository sb3-contrib[49]. SB3 is well-suited for research and even some production use for controlling smaller models or environments, but it isn't designed for LLM-scale models out of the box.
- **Ray RLlib:** A scalable RL library built on Ray, which can distribute training across clusters easily. RLlib supports PPO (and many other algorithms) and can handle large-scale simulations. It's more focused on environments and simulators but in theory could be used for RLHF with custom environments. RLlib provides high-level APIs to scale experience collection on many workers, which could be repurposed for large language model fine-tuning tasks as well.
- **Hugging Face TRL (Transformer Reinforcement Learning):** This library is specifically tailored for applying RL (especially RLHF) to transformers and LLMs. It provides implementations of PPO, as well as newer algorithms like DPO and GRPO for language model fine-tuning[38]. For example, `TRL.GRPOTrainer` is an out-of-the-box trainer for GRPO on language models[38]. It abstracts away the complexity of managing generation of groups, reward computation, etc., making it much easier to try GRPO on your model. TRL also integrates with Hugging Face's Accelerate and DeepSpeed to handle distributed training on multiple GPUs/nodes, and provides useful features for logging, evaluation, and dealing with large models. This is currently one of the best frameworks for RLHF on LLMs, as it keeps up with the latest research (like supporting new algorithms such as P3O, Rejection sampling, etc., aside from PPO/GRPO).
- **DeepSpeed & DeepSpeed-Chat:** DeepSpeed (by Microsoft) is a deep learning optimization library that also has a module for RLHF called DeepSpeed-Chat. It provides pipeline parallelism, memory optimizations (ZeRO), and some RLHF training pipeline components, potentially using PPO under the hood. While not an RL library per se, it's a tool to efficiently train large models (including the RLHF stage) on clusters.
- **Anthropic's TRLX (CarperAI):** TRLX is an open-source library for RLHF which originally provided a PPO implementation for language models (CarperAI's version of PPO with some tricks). It’s somewhat similar to HF TRL but with a different API. It might not yet include GRPO, but it was used for some early open-source RLHF experiments.
- **Research code for new algorithms:** For GRPO, aside from HuggingFace TRL, the reference implementation was used in the DeepSeekMath project. As GRPO is new, it's a good idea to rely on these implementations rather than writing it entirely from scratch, to ensure subtle details (like how exactly KL is computed, or how reward scaling is done) are handled as in the paper.

When choosing a framework, consider your use case: for standard RL tasks (like controlling an agent in a simulator), Stable Baselines3 or RLlib with PPO is a solid choice. For LLM fine-tuning with RLHF, Hugging Face's TRL is more specialized and provides the tools needed for language generation scenarios (including models, tokenizers integration, etc.). In all cases, leveraging these libraries can save a lot of time and ensure you use validated implementations.

## Conclusion

Reinforcement learning has become an integral part of training large language models to align them with desired behaviors. We examined TRPO, PPO, and GRPO – three generations of policy optimization algorithms – and saw how each builds upon the last to improve either stability or efficiency (or both).

- **TRPO** introduced the idea of cautious updates with a trust region, which was critical for early successes in stable RL training[50]. However, its complexity and computational burden make it less practical for today's enormous models.
- **PPO** simplified those ideas, making advanced RL training widely accessible and effective; it remains the workhorse for many applications[50].
- **GRPO** took it a step further by addressing the specific hardware and scale challenges of LLM fine-tuning, removing the need for a value network and leveraging the strengths of language models (the ability to generate many samples) to both improve performance and drastically cut resource usage[51].

The evolution from TRPO → PPO → GRPO illustrates a general trend in ML: algorithms need to be not just theoretically sound, but also hardware-friendly and scalable. GRPO’s design was directly motivated by the practical constraints of training huge models on GPU clusters, and it exemplifies how new research is focusing on making RL more efficient in the large-scale regime.

Finally, RL is becoming more important in the training pipeline of AI models because it enables optimizing for what we truly care about. Pre-training on internet data and supervised fine-tuning get you a strong base model, but reinforcement learning allows you to define a reward signal for quality as you want it and directly tune the model for that. Whether it's user satisfaction, factual correctness, safety, or reasoning ability, RL provides a framework to continue improving a model based on feedback and desired outcomes, rather than just next-token prediction. As one author noted, “Reinforcement Learning will become the main focus in training LLMs to improve their performance, surpassing pre-training and SFT in driving future innovations.”[3] With algorithms like PPO and GRPO, we now have the tools to do this at scale.

## References

The content above is drawn from a mix of research papers and expert commentary. For further reading, consider OpenAI’s PPO paper (Schulman et al. 2017) for background, the DeepSeekMath paper (Shao et al. 2024) for GRPO details, the HuggingFace TRL documentation[38] for practical implementation tips, and the RLHF literature (e.g., the RLHF Book by N. Lambert) for a deeper theoretical understanding of these algorithms in the context of language model fine-tuning. All in all, mastering these algorithms and their implementations will equip you to push the frontier of what large language models can do through reinforcement learning.

- [1] [2] [3] [5] [7] [12] [15] [17] [18] [29] [32] [34] [48] [50] [51] [Training Large Language Models: From TRPO to GRPO | by Maxime Wolf | Data Science Collective | Medium](https://medium.com/data-science-collective/training-large-language-models-from-trpo-to-grpo-f3607a126194)
- [4] [6] [8] [9] [10] [11] [16] [21] [Trust Region Policy Optimization (TRPO) | by Leonidas Gorgo | Nov, 2025 | Medium](https://leonidasgorgo.medium.com/trust-region-policy-optimization-trpo-d9f5536d6aeb)
- [13] [14] [25] [26] [27] [28] [33] [35] [36] [37] [38] [39] [40] [41] [42] [43] [46] [GRPO vs Other RL Algorithms: A Simple, Clear Guide](https://company.hpc-ai.com/blog/grpo-vs-other-rl-algorithms-a-simple-clear-guide)
- [19] [20] [22] [31] [45] [Reinforcement Learning (i.e. Policy Gradient Algorithms) | RLHF Book by Nathan Lambert](https://rlhfbook.com/c/11-policy-gradients.html)
- [23] [24] [30] [44] [47] [GRPO Trainer](https://huggingface.co/docs/trl/main/en/grpo_trainer)
- [49] [Reinforcement Learning Tips and Tricks - Stable Baselines3](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html)